# This notebook will run replicates of a model and perform a random search of hyperparameters

### Imports

In [1]:
import data_handling
import training_and_validation
import hyperparam_tuning

In [2]:
#imports
import importlib
import glob

from models import PD_CNN, ResNet, ResidualBlock, PD_LSTM
import torch
import training_and_validation
from training_and_validation import loso_cross_validation, train, validate, cross_train, train_with_validation
from result_visualization import run_chi_squared_test, plot_confusion_matrix
from torch.utils.data import DataLoader, random_split
import data_handling
import models
import result_visualization
import random
import hyperparam_tuning
import os
from sys import platform

### Find training data, make dataset

In [3]:
#set the device to cuda:0 
if torch.cuda.is_available():  
  device = "cuda:0" 
else:  
  device = "cpu" 
device

'cpu'

In [4]:
#please specify where you keep your training dataset
training_data_src =  './Data/UNM/small_subset/'

############ create list of subject numbers to leave out ###############################
files = glob.glob(training_data_src + '*.csv')
leave_one_out_list = []
for file in files:  
  if platform=='win32':
    leave_one_out_list.append(file.split('\\')[-1])#.split('_')[1]) #remove hashtags to return to UNM dataset
  elif platform=='linux' or platform=='linux2' or platform.startswith('darwin'):
    leave_one_out_list.append(file.split('/')[-1])
  else:
    assert ValueError, 'operating system not identified'

In [7]:

#make training dataset
if device == 'cuda:0':
    print('making a TensorDataset')
    training_dataset = data_handling.make_data_into_tensor(data_path=training_data_src, device=device, chunk_size=512)
else:
    print('making a custom Dataset')
    training_dataset = data_handling.EEGDataset(data_path=training_data_src, chunk_size=512)

making a custom Dataset
there are this many items in the list of data  583
there are this many items in the list of labels  583
The length of the lists of channels means and stds is  60


### Choose model and search space

In [9]:
experiment_model = 'Transformer'
experiment_name = experiment_model+'adapting_search'
training_target_dir = './training_results/'+experiment_model+'/'+experiment_name+'/'

In [ ]:
importlib.reload(hyperparam_tuning)
importlib.reload(training_and_validation)
importlib.reload(models)
import models
import training_and_validation
import hyperparam_tuning
#run cross validation for N random configurations of hyperparameters
hyperparam_tuning.perform_random_hyperparameter_search(training_dataset, leave_one_out_list, sample_size=2,
                                                       learning_rate_min_max=(0.00001,0.001), batch_min_max=(2,32), 
                                                       attention_blocks=6, heads=4, model_dim=60, seq_length=512,
                                                       search_title=experiment_name, save_path=training_target_dir,
                                                       model_type=experiment_model, device=device) #13 worked for one epoch